In [6]:
# from softcomputing_packages_bu import *
from apyori import apriori
import pandas as pd
import numpy as np
import copy

import networkx as nx
import matplotlib.pyplot as plt

In [7]:
# Load dataset
hc_csv_raw = pd.read_csv('./datasets/2.0-discretized-v2-2-2-with-label-resampled-and-DLP-encoded.csv')
hc_csv_raw.head()
len(hc_csv_raw)

41673

In [8]:
# Sampling (for Development)
n_sample = 30000
hc_csv = hc_csv_raw.sample(frac=1, random_state=42)
hc_csv = hc_csv.iloc[:n_sample]
hc_csv.head()

,global_active_power,global_reactive_power,voltage,global_intensity,kitchen,laundry,climate_control,other,weekend,month_name,season_name,day_name,peak_label_pred,attention_type_label,activation_type_label,peak_warning,no_significant_change,lower_than_usual
35103,Medium,Very High,Very High,Medium,Medium,Low,High,High,True,april,spring,sunday,lower_than_usual,other,cam_2,False,False,True
8049,Low,High,Very High,Low,Low,Low,Low,Medium,False,march,spring,tuesday,no_significant_change,other,cam_2,False,True,False
22355,Low,Medium,Medium,Low,Medium,High,Medium,Low,False,september,autumn,friday,peak_warning,other,cam_2,True,False,False
21779,Medium,Very High,Medium,Medium,Very High,Very High,High,Medium,True,june,summer,saturday,peak_warning,early_attention,cam_1,True,False,False
40302,Low,Medium,High,Low,High,High,Medium,Medium,False,march,spring,wednesday,lower_than_usual,other,cam_2,False,False,True


In [9]:
# Preprocess dataset
for c in range(8):
    print(c)
    print("[%d] Column Preprocessing: %s" %(c, hc_csv.columns[c]))
    for r in range(hc_csv.shape[0]):
        if(hc_csv.iloc[r,c]=='Very High'):
            hc_csv.iloc[r,c] = hc_csv.columns[c]+' 75-100%'
        elif(hc_csv.iloc[r,c]=='High'):
            hc_csv.iloc[r,c] = hc_csv.columns[c]+' 50-75%'
        elif(hc_csv.iloc[r,c]=='Medium'):
            hc_csv.iloc[r,c] = hc_csv.columns[c]+' 25-50%'
        elif(hc_csv.iloc[r,c]=='Low'):
            # hc_csv.iloc[r,c] = '-1'
            hc_csv.iloc[r,c] = hc_csv.columns[c]+' 0-25%'
        else:
            hc_csv.iloc[r,c] = '-1'
print("Done.")

0
[0] Column Preprocessing: global_active_power
1
[1] Column Preprocessing: global_reactive_power
2
[2] Column Preprocessing: voltage
3
[3] Column Preprocessing: global_intensity
4
[4] Column Preprocessing: kitchen
5
[5] Column Preprocessing: laundry
6
[6] Column Preprocessing: climate_control
7
[7] Column Preprocessing: other
Done.


In [10]:
for c in hc_csv.columns:
  if c == 'weekend':
    for r in range(hc_csv.shape[0]):
      if hc_csv.iloc[r,list(hc_csv.columns).index('weekend')] == True:
        hc_csv.iloc[r,list(hc_csv.columns).index('weekend')] = 'weekend True'
      else:
        hc_csv.iloc[r,list(hc_csv.columns).index('weekend')] = 'weekend False'
      # break

C:\Users\sclab\AppData\Local\Temp\ipykernel_37660\2109154175.py:5: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'weekend True' has dtype incompatible with bool, please explicitly cast to a compatible dtype first.
  hc_csv.iloc[r,list(hc_csv.columns).index('weekend')] = 'weekend True'


In [11]:
hc_csv.sample(5)

,global_active_power,global_reactive_power,voltage,global_intensity,kitchen,laundry,climate_control,other,weekend,month_name,season_name,day_name,peak_label_pred,attention_type_label,activation_type_label,peak_warning,no_significant_change,lower_than_usual
3393,global_active_power 50-75%,global_reactive_power 50-75%,voltage 75-100%,global_intensity 50-75%,kitchen 50-75%,laundry 25-50%,climate_control 50-75%,other 50-75%,weekend False,march,spring,thursday,no_significant_change,other,cam_2,False,True,False
38076,global_active_power 0-25%,global_reactive_power 0-25%,voltage 50-75%,global_intensity 0-25%,kitchen 75-100%,laundry 25-50%,climate_control 50-75%,other 25-50%,weekend False,april,spring,thursday,lower_than_usual,early_attention,cam_2,False,False,True
28184,global_active_power 25-50%,global_reactive_power 0-25%,voltage 25-50%,global_intensity 25-50%,kitchen 25-50%,laundry 0-25%,climate_control 50-75%,other 0-25%,weekend False,september,autumn,thursday,lower_than_usual,other,cam_2,False,False,True
35419,global_active_power 0-25%,global_reactive_power 0-25%,voltage 75-100%,global_intensity 0-25%,kitchen 0-25%,laundry 0-25%,climate_control 0-25%,other 0-25%,weekend False,december,winter,friday,lower_than_usual,late_attention,cam_2,False,False,True
2525,global_active_power 25-50%,global_reactive_power 25-50%,voltage 50-75%,global_intensity 25-50%,kitchen 50-75%,laundry 25-50%,climate_control 25-50%,other 50-75%,weekend True,january,winter,saturday,no_significant_change,late_attention,cam_2,False,True,False


In [12]:
# targets_only = ['peak_label_pred', 'no_significant_change', 'lower_than_usual']

In [13]:
hc_csv_temp = copy.copy(hc_csv)
hc_csv_temp.drop(columns=['peak_label_pred', 'no_significant_change', 'peak_warning', 'month_name', 'day_name'], inplace=True)

In [14]:
for i in range(len(hc_csv_temp)):
    if hc_csv_temp.iloc[i,-1] == True:
        hc_csv_temp.iloc[i,-1] = 'True'
    else:
        hc_csv_temp.iloc[i,-1] = 'False'
    # break

C:\Users\sclab\AppData\Local\Temp\ipykernel_37660\3489011138.py:3: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'True' has dtype incompatible with bool, please explicitly cast to a compatible dtype first.
  hc_csv_temp.iloc[i,-1] = 'True'


In [15]:
# hc_csv_temp.iloc[:,12].name

for i in range(len(hc_csv_temp)):
    hc_csv_temp.iloc[i,9] = hc_csv_temp.iloc[:,9].name + " " + hc_csv_temp.iloc[i,9]
    
for i in range(len(hc_csv_temp)):
    hc_csv_temp.iloc[i,-3] = hc_csv_temp.iloc[:,-3].name + " " + hc_csv_temp.iloc[i,-3]

In [16]:
# Season
for i in range(len(hc_csv_temp)):
    hc_csv_temp.iloc[i,9] = hc_csv_temp.iloc[:,9].name + " " + hc_csv_temp.iloc[i,9]

In [17]:
hc_csv_temp.head(4)

,global_active_power,global_reactive_power,voltage,global_intensity,kitchen,laundry,climate_control,other,weekend,season_name,attention_type_label,activation_type_label,lower_than_usual
35103,global_active_power 25-50%,global_reactive_power 75-100%,voltage 75-100%,global_intensity 25-50%,kitchen 25-50%,laundry 0-25%,climate_control 50-75%,other 50-75%,weekend True,season_name season_name spring,attention_type_label other,cam_2,True
8049,global_active_power 0-25%,global_reactive_power 50-75%,voltage 75-100%,global_intensity 0-25%,kitchen 0-25%,laundry 0-25%,climate_control 0-25%,other 25-50%,weekend False,season_name season_name spring,attention_type_label other,cam_2,False
22355,global_active_power 0-25%,global_reactive_power 25-50%,voltage 25-50%,global_intensity 0-25%,kitchen 25-50%,laundry 50-75%,climate_control 25-50%,other 0-25%,weekend False,season_name season_name autumn,attention_type_label other,cam_2,False
21779,global_active_power 25-50%,global_reactive_power 75-100%,voltage 25-50%,global_intensity 25-50%,kitchen 75-100%,laundry 75-100%,climate_control 50-75%,other 25-50%,weekend True,season_name season_name summer,attention_type_label early_attention,cam_1,False


In [18]:
# hc_csv to list
hc_list = []
for r in range(hc_csv_temp.shape[0]):
    rule_buffer = []
    for c in range(len(hc_csv_temp.iloc[r])):
        if(hc_csv_temp.iloc[r,c]!='-1'):
            rule_buffer.append(hc_csv_temp.iloc[r,c])
    hc_list.append(rule_buffer)

In [19]:
hc_list

[['global_active_power 25-50%',
  'global_reactive_power 75-100%',
  'voltage 75-100%',
  'global_intensity 25-50%',
  'kitchen 25-50%',
  'laundry 0-25%',
  'climate_control 50-75%',
  'other 50-75%',
  'weekend True',
  'season_name season_name spring',
  'attention_type_label other',
  'cam_2',
  'True'],
 ['global_active_power 0-25%',
  'global_reactive_power 50-75%',
  'voltage 75-100%',
  'global_intensity 0-25%',
  'kitchen 0-25%',
  'laundry 0-25%',
  'climate_control 0-25%',
  'other 25-50%',
  'weekend False',
  'season_name season_name spring',
  'attention_type_label other',
  'cam_2',
  'False'],
 ['global_active_power 0-25%',
  'global_reactive_power 25-50%',
  'voltage 25-50%',
  'global_intensity 0-25%',
  'kitchen 25-50%',
  'laundry 50-75%',
  'climate_control 25-50%',
  'other 0-25%',
  'weekend False',
  'season_name season_name autumn',
  'attention_type_label other',
  'cam_2',
  'False'],
 ['global_active_power 25-50%',
  'global_reactive_power 75-100%',
  'volta

In [20]:
# Association Rule Mining (ARM) (Caution: extremly slow)
association_rules = list(apriori(hc_list, min_support=0.05, min_confidence=0.2))
print('Rule Mined: %d' %(len(association_rules)))

Rule Mined: 3529


In [21]:
# Rules to numpy structure
association_rules_np = []
for r in range(len(association_rules)):
    rule_buffer = []
    association_rule = association_rules[r]
    items = [x for x in association_rule[0]]
    n_items = len(items)
    support = association_rule[1]
    confidence = association_rule[2][0][2]
    lift = association_rule[2][0][3]
    rule_buffer.append(n_items), rule_buffer.append(np.array(items)), rule_buffer.append(round(support, 4)), rule_buffer.append(round(confidence, 4)), rule_buffer.append(round(lift, 4))
    print(rule_buffer)
    association_rules_np.append(rule_buffer)
# Fix later
# association_rules_np = np.array(association_rules_np)
# print(association_rules_np)
# np.save('./association_rules', association_rules_np)
# Length, Rule, Support, Confidence, Lift

[1, array(['False'], dtype='<U5'), 0.6663, 0.6663, 1.0]
[1, array(['True'], dtype='<U4'), 0.3337, 0.3337, 1.0]
[1, array(['attention_type_label early_attention'], dtype='<U36'), 0.2196, 0.2196, 1.0]
[1, array(['attention_type_label late_attention'], dtype='<U35'), 0.2728, 0.2728, 1.0]
[1, array(['attention_type_label other'], dtype='<U26'), 0.5076, 0.5076, 1.0]
[1, array(['cam_1'], dtype='<U5'), 0.3932, 0.3932, 1.0]
[1, array(['cam_2'], dtype='<U5'), 0.6068, 0.6068, 1.0]
[1, array(['climate_control 0-25%'], dtype='<U21'), 0.3145, 0.3145, 1.0]
[1, array(['climate_control 25-50%'], dtype='<U22'), 0.2476, 0.2476, 1.0]
[1, array(['climate_control 50-75%'], dtype='<U22'), 0.2261, 0.2261, 1.0]
[1, array(['climate_control 75-100%'], dtype='<U23'), 0.2118, 0.2118, 1.0]
[1, array(['global_active_power 0-25%'], dtype='<U25'), 0.3074, 0.3074, 1.0]
[1, array(['global_active_power 25-50%'], dtype='<U26'), 0.2678, 0.2678, 1.0]
[1, array(['global_active_power 50-75%'], dtype='<U26'), 0.2061, 0.2061, 

In [22]:
# Filter by rhs=='Power Pattern *'
filter_RHS_ECP = True
if(filter_RHS_ECP):
    association_rules_np_rhs = []
    for r in range(len(association_rules_np)):
        # rhs = association_rules_np[r, 1][-1] # old
        rhs = association_rules_np[r][1][-1]
        if(rhs=='True' or rhs=='False'):
            association_rules_np_rhs.append(association_rules_np[r])
    association_rules_np_rhs = np.array(association_rules_np_rhs, dtype=object)
else:
    association_rules_np_rhs = association_rules_np
print('Rule Filtered: %d' %(len(association_rules_np_rhs)))

Rule Filtered: 69


In [23]:
# Sort rules by confidence
association_rules_np_rhs_sorted = association_rules_np_rhs[association_rules_np_rhs[:, 3].argsort()][::-1]
print(len(association_rules_np_rhs_sorted))
print(association_rules_np_rhs_sorted[:10])

# Length, Rule, Support, Confidence, Lift

69
[[1 array(['False'], dtype='<U5') 0.6663 0.6663 1.0]
 [2 array(['attention_type_label other', 'True'], dtype='<U26') 0.1948
  0.5837 1.1498]
 [2 array(['global_intensity 0-25%', 'True'], dtype='<U22') 0.1849 0.5541
  1.7626]
 [3
  array(['attention_type_label late_attention', 'global_intensity 0-25%',
         'True'], dtype='<U35')
  0.0512 0.5374 1.7095]
 [2 array(['voltage 75-100%', 'True'], dtype='<U15') 0.1746 0.5231 1.7337]
 [2 array(['other 25-50%', 'True'], dtype='<U12') 0.1509 0.4523 1.5595]
 [5 array(['cam_2', 'weekend False', 'global_intensity 0-25%',
           'attention_type_label other', 'False'], dtype='<U26') 0.0551
  0.4256 1.6496]
 [2 array(['voltage 75-100%', 'False'], dtype='<U15') 0.1272 0.4215
  0.6325]
 [5 array(['cam_2', 'weekend False', 'global_active_power 0-25%',
           'attention_type_label other', 'False'], dtype='<U26')  0.0554
  0.4196 1.6265]
 [5
  array(['weekend False', 'global_intensity 0-25%',
         'global_active_power 0-25%', 'attention_

In [31]:
# Duplicate Filter

# Filter by rhs=='No change - True'
top_k = 10

filtered_rules_individual = set([])
filtered_rules = []

# rule_states = set({'attention_type_label other'})
# rule_states = set({})
rule_counter = {}
association_rules_np_rhs_buffer = []
for r in range(len(association_rules_np_rhs_sorted)):
    rhs = association_rules_np_rhs_sorted[r, 1][-1]
    if(rhs=='True' and association_rules_np_rhs_sorted[r, 0]>2 and association_rules_np_rhs_sorted[r, -1]>1.5):
        association_rules_np_rhs_buffer.append(association_rules_np_rhs_sorted[r])
association_rules_np_rhs_buffer = np.array(association_rules_np_rhs_buffer)

# print(association_rules_np_rhs_buffer[:top_k])
for rule in association_rules_np_rhs_buffer:
    
    cari_lagi = False
    # rule_checker_vars_only = set({})
    rule = rule[1][:-1].tolist()
    # print(rule)
    if top_k < 1:
        break
    
    print('mined rule: ', rule)
    
    # top_k = top_k - 1
    filtered_rules_individual_temp = filtered_rules_individual | set(rule)
    # filtered_rules_individual = filtered_rules_individual | set(rule)
    if (len(filtered_rules_individual_temp) > len(filtered_rules_individual)):
        rule_checker_vars_only = set([r.split(' ')[0] for r in filtered_rules_individual_temp])
        print('ada tambahan! cek length')
        print('set dgn tambahan', filtered_rules_individual_temp)
        print('set vars only', rule_checker_vars_only)
        if (len(rule_checker_vars_only) == len(filtered_rules_individual_temp)):
            print('element berbeda dan belum pernah dimasukan!')
        else:
            print('elemen ada yang kembar~ cek lagi yang lain')
            cari_lagi = True
        # break
    if (cari_lagi):
        continue
    
    # jika udah klop, copy aja
    filtered_rules_individual = filtered_rules_individual | set(rule)
    filtered_rules.append(rule)
    top_k = top_k - 1
    print('filtered rules', filtered_rules)
    print('filtered individual', filtered_rules_individual)
    print('next search')
    print()
# filtered_rules_individual
# filtered_rules
'Filtered rules for peak warning',(filtered_rules_individual), filtered_rules, len(filtered_rules)

mined rule:  ['attention_type_label late_attention', 'global_intensity 0-25%']
ada tambahan! cek length
set dgn tambahan {'attention_type_label late_attention', 'global_intensity 0-25%'}
set vars only {'attention_type_label', 'global_intensity'}
element berbeda dan belum pernah dimasukan!
filtered rules [['attention_type_label late_attention', 'global_intensity 0-25%']]
filtered individual {'attention_type_label late_attention', 'global_intensity 0-25%'}
next search

mined rule:  ['global_intensity 0-25%', 'attention_type_label other']
ada tambahan! cek length
set dgn tambahan {'attention_type_label late_attention', 'global_intensity 0-25%', 'attention_type_label other'}
set vars only {'attention_type_label', 'global_intensity'}
elemen ada yang kembar~ cek lagi yang lain
mined rule:  ['voltage 75-100%', 'attention_type_label other']
ada tambahan! cek length
set dgn tambahan {'voltage 75-100%', 'attention_type_label late_attention', 'global_intensity 0-25%', 'attention_type_label other'

('Filtered rules for peak warning',
 {'attention_type_label late_attention',
  'global_active_power 25-50%',
  'global_intensity 0-25%',
  'kitchen 0-25%',
  'other 25-50%',
  'voltage 75-100%'},
 [['attention_type_label late_attention', 'global_intensity 0-25%'],
  ['voltage 75-100%', 'global_intensity 0-25%'],
  ['voltage 75-100%', 'other 25-50%'],
  ['other 25-50%', 'global_intensity 0-25%'],
  ['kitchen 0-25%', 'global_intensity 0-25%'],
  ['voltage 75-100%', 'global_active_power 25-50%'],
  ['voltage 75-100%', 'other 25-50%', 'global_intensity 0-25%']],
 7)

OLD V2

In [290]:
# Filter by rhs=='No change - True'
top_k = 10

filtered_rules_individual = []
filtered_rules = []

# rule_states = set({'attention_type_label other'})
# rule_states = set({})
rule_counter = {}
association_rules_np_rhs_buffer = []
for r in range(len(association_rules_np_rhs_sorted)):
    rhs = association_rules_np_rhs_sorted[r, 1][-1]
    if(rhs=='True' and association_rules_np_rhs_sorted[r, 0]>1):
        association_rules_np_rhs_buffer.append(association_rules_np_rhs_sorted[r])
association_rules_np_rhs_buffer = np.array(association_rules_np_rhs_buffer)

# print(association_rules_np_rhs_buffer[:top_k])
for rule in association_rules_np_rhs_buffer:
    
    # print(rule)
    if top_k < 1:
        break
    if rule[-1] < 1:
        continue
    print('mined rule: ', rule[1][:-1].tolist())
    
    # # cek masing2 rule di rule[1], apakah ada duplikatnya atau enggak
    # save = True
    # for r in rule[1].tolist():
    #     # rule_states_vars = [i.split(' ')[0] for i in rule_states]
    #     rule_states_temp = rule_states | {r}
    #     # rule_states = rule_states | {r}
    #     # print(rule_states_temp)
    #     len_tmp = [i.split(' ')[0] for i in rule_states_temp]
    #     len_ori = [i.split(' ')[0] for i in rule_states]
    #     print(len_tmp, len_ori)
    #     print(len(len_tmp), len(len_ori))
    #     if (len(len_tmp) == len(len_ori)):
    #     # if (len(rule_states_temp) == len(rule_states)):
    #         print('Duplicate found', r)
    #         save = False
    #         continue
        
    # if save == False:
    #     continue
    # rule_states = rule_states | 
    top_k = top_k - 1
    filtered_rules_individual.extend(rule[1][:-1].tolist())
    filtered_rules.append(rule[1][:-1].tolist())
    print('filtered rules', filtered_rules)
    print('next search')
    print()
# filtered_rules_individual
filtered_rules
set(filtered_rules_individual) - {'True'}

mined rule:  ['attention_type_label other']
filtered rules [['attention_type_label other']]
next search

mined rule:  ['activation_type_label cam_2', 'weekend False']
filtered rules [['attention_type_label other'], ['activation_type_label cam_2', 'weekend False']]
next search

mined rule:  ['weekend False', 'attention_type_label other']
filtered rules [['attention_type_label other'], ['activation_type_label cam_2', 'weekend False'], ['weekend False', 'attention_type_label other']]
next search

mined rule:  ['activation_type_label cam_2', 'attention_type_label other']
filtered rules [['attention_type_label other'], ['activation_type_label cam_2', 'weekend False'], ['weekend False', 'attention_type_label other'], ['activation_type_label cam_2', 'attention_type_label other']]
next search

mined rule:  ['global_active_power 25-50%']
filtered rules [['attention_type_label other'], ['activation_type_label cam_2', 'weekend False'], ['weekend False', 'attention_type_label other'], ['activation

{'activation_type_label cam_2',
 'attention_type_label other',
 'global_active_power 0-25%',
 'global_active_power 25-50%',
 'global_active_power 50-75%',
 'global_intensity 0-25%',
 'global_intensity 25-50%',
 'global_intensity 50-75%',
 'weekend False'}

OLD

In [167]:
# Filter by rhs=='No change - True'
top_k = 10

filtered_rules = []
rule_states = set({})
rule_counter = {}
association_rules_np_rhs_buffer = []
for r in range(len(association_rules_np_rhs_sorted)):
    rhs = association_rules_np_rhs_sorted[r, 1][-1]
    if(rhs=='True' and association_rules_np_rhs_sorted[r, 0]>1):
        association_rules_np_rhs_buffer.append(association_rules_np_rhs_sorted[r])
association_rules_np_rhs_buffer = np.array(association_rules_np_rhs_buffer)
# print(association_rules_np_rhs_buffer[:top_k])
for rule in association_rules_np_rhs_buffer:
    # print(rule)
    if top_k < 1:
        break
    if rule[0] == 0 or rule[-1] < 1:
        continue
    
    print('Mined rule: ',rule)
    # rule_states = rule_states | (set(rule[1].tolist()) - {'True'})
    break_inside = False
    for r in set(rule[1].tolist()) - {'True'}:
        if r not in rule_counter:
            if (len(rule_counter) == False):
                rule_counter[r] = 1
            else:
                rules = rule_counter.keys()
                # cek data duplikat ngga
                rule_counter_temp = rule_counter.copy()
                rule_counter_temp[r] = 1
                fr = set(rule_counter_temp.keys()) - {'True'}
                # fr
                frs = [r.split(" ")[0] for r in fr]
                print('copy rule', rule_counter_temp)
                # if (len(frs) == )
                print('frs ', frs, list(set(frs)))
                # if set is same, then no duplicate
                if len(frs) != len(list(set(frs))):
                    print('found duplicate')
                    break_inside = True
                    break
                # print()
                rule_counter[r] = 1
        else:
            rule_counter[r] += 1
    if break_inside:
        continue
    filtered_rules.extend(rule[1].tolist())
    
    top_k = top_k - 1
    print('Found! k=', top_k)
    print('Filtered rules: ', filtered_rules)
    print('Rules counter: ', rule_counter)
    print('rule:', rule[1].tolist())
    print('')

print('unique vars')
print(set(filtered_rules) - {'True'})
print(rule_states)
print('Rule counter:')
print(rule_counter)

Mined rule:  [2 array(['attention_type_label other', 'True'], dtype='<U26') 0.1714
 0.513 1.0107]
Found! k= 9
Filtered rules:  ['attention_type_label other', 'True']
Rules counter:  {'attention_type_label other': 1}
rule: ['attention_type_label other', 'True']

Mined rule:  [3 array(['activation_type_label cam_2', 'weekday', 'True'], dtype='<U27')
 0.1644 0.4921 1.1226]
copy rule {'attention_type_label other': 1, 'activation_type_label cam_2': 1}
frs  ['activation_type_label', 'attention_type_label'] ['attention_type_label', 'activation_type_label']
copy rule {'attention_type_label other': 1, 'activation_type_label cam_2': 1, 'weekday': 1}
frs  ['activation_type_label', 'attention_type_label', 'weekday'] ['attention_type_label', 'activation_type_label', 'weekday']
Found! k= 8
Filtered rules:  ['attention_type_label other', 'True', 'activation_type_label cam_2', 'weekday', 'True']
Rules counter:  {'attention_type_label other': 1, 'activation_type_label cam_2': 1, 'weekday': 1}
rule: ['a

['other',
 'global_intensity 0-25%',
 'True',
 'other',
 'global_active_power 0-25%',
 'True',
 'other',
 'climate_control 0-25%',
 'True',
 'other',
 'True',
 'cam_2',
 'weekday',
 'True',
 'other',
 'weekday',
 'True',
 'cam_2',
 'other',
 'True',
 'monday',
 'True',
 'weekday',
 'monday',
 'True',
 'other',
 'voltage 75-100%',
 'True']

In [125]:
dup = ['other', 'climate_control', 'monday', 'global_active_power', 'cam_2', 'global_intensity', 'weekday'] 
# ['other', 'global_active_power', 'monday', 'climate_control', 'cam_2', 'weekday', 'global_intensity']
len(set(dup)), len(dup) # (7, 8) duplicate
len(set(dup)), len(dup) # (7, 7) aman banget

(7, 7)

In [104]:
rule_counter = {'other': 1, 'climate_control 0-25%':1, 'monday':1, 'global_active_power 25-50%':1, 'cam_2':1, 'global_intensity 0-25%':1, 'weekday':1, 'global_active_power 0-25%':1}
list(rule_counter.keys())

['other',
 'climate_control 0-25%',
 'monday',
 'global_active_power 25-50%',
 'cam_2',
 'global_intensity 0-25%',
 'weekday',
 'global_active_power 0-25%']

In [106]:
fr = set(rule_counter.keys()) - {'True'}
# fr
frs = [r.split(" ")[0] for r in fr]

frs, list(set(frs))

(['other',
  'climate_control',
  'monday',
  'global_active_power',
  'cam_2',
  'global_intensity',
  'weekday',
  'global_active_power'],
 ['other',
  'global_active_power',
  'monday',
  'climate_control',
  'cam_2',
  'weekday',
  'global_intensity'])